In [16]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import warnings
warnings.filterwarnings('ignore')

In [17]:
import tensorflow
from tensorflow.python.client import device_lib
from  keras.models import Sequential, Model
from  keras.layers import Input, Dense, LeakyReLU, Activation, Concatenate, Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D, InputLayer, Flatten, BatchNormalization, Reshape, Lambda
from  keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau 
from  keras.optimizers import RMSprop, Adam
from  keras.preprocessing.image import load_img, ImageDataGenerator
from  keras.utils import multi_gpu_model
from IPython.display import Image 

import keras_metrics
import pandas
import ast
import numpy as np
import matplotlib.patches as patches 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image, ImageDraw,ImageFont
import os
import sys
import subprocess
#device_lib.list_local_devices()

In [18]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import keras.backend as K
import numpy as np
import random

In [19]:
def yolf_loss(y_true, y_pred):
    #true e pred sono [32][245]
    
    #tensorflow.print("tensors:", y_pred)
    #tensorflow.print("batch_size:", tensorflow.shape(y_pred)[0])
    
    #y_pred = tensorflow.reshape(y_pred, [tensorflow.shape(y_pred)[0], 7, 7, 5])
    #y_true = tensorflow.reshape(y_true, [tensorflow.shape(y_pred)[0], 7, 7, 5])


    b_p_pred = y_pred[:,0]
    b_x_pred = y_pred[:,1]
    b_y_pred = y_pred[:,2]
    b_w_pred = y_pred[:,3]
    b_h_pred = y_pred[:,4]


    b_p = y_true[:,0]
    b_x = y_true[:,1]
    b_y = y_true[:,2]
    b_w = y_true[:,3]
    b_h = y_true[:,4]
    #tensorflow.print(b_p_pred)
    #print(b_p)
    #print(b_x, b_x_pred)
    #print(b_xy_pred.get_shape(),b_xy.get_shape())
    #print(b_wh_pred.get_shape(),b_wh.get_shape())
    #indicator_coord = K.expand_dims(y_true[ 3], axis=-1) * 1.0
    #loss_p = K.mean(K.square(b_p - b_p_pred), axis=-1)
    loss_xy = K.sum(b_p * (K.square(b_x - b_x_pred) + K.square(b_y - b_y_pred)))# * indicator_coord)#, axis=[1,2,3,4])
    
    #b_w = K.sqrt(b_w)
    #b_h = K.sqrt(b_h)
    #b_w_pred = K.sqrt(b_w_pred)
    #b_h_pred = K.sqrt(b_h_pred)
    
    loss_wh = K.sum(
        b_p * 
        (
            (K.square(b_w - b_w_pred)) + 
            (K.square(b_h - b_h_pred))
        ))# * indicator_coord)#, axis=[1,2,3,4])

    #tensorflow.print("loss_p:", loss_p)
    #tensorflow.print("loss_xy:", loss_xy)
    #tensorflow.print("loss_wh:", loss_wh)

    #print(K.cast(loss_p, dtype="float32"))
    #print(K.cast(loss_xy, dtype="float32"))
    #print(loss_wh)
    #tensorflow.print("Loss:", ( K.cast(loss_p, dtype="float32") + loss_wh +  K.cast(loss_xy, dtype="float32") )/3)
    return (loss_p + loss_wh +  K.cast(loss_xy, dtype="float32"))

for el in train_generator:
    #print(el)
    yolf_loss(el[1], el[1])
    break

In [20]:
BATCH_SIZE = 128
IMG_SIZE = 224

In [21]:
train_df = pandas.read_csv("/data01/ML/dataset/FACE_CLASSIFIER/train2.csv")
valid_df = pandas.read_csv("/data01/ML/dataset/FACE_CLASSIFIER/val2.csv")
test_df = pandas.read_csv("/data01/ML/dataset/FACE_CLASSIFIER/test2.csv")

def convert_paths(path_string):
    temp = path_string.replace("\\", "/")  
    return "/data01/ML/" + temp.split("/",1)[1]

train_df["image_path"] = train_df["image_path"].apply(convert_paths)
valid_df["image_path"] = valid_df["image_path"].apply(convert_paths)
test_df["image_path"] = test_df["image_path"].apply(convert_paths)

In [22]:
print(train_df.iloc[0]["image_path"])

/data01/ML/dataset/FACE_CLASSIFIER/256_ObjectCategories/019.boxing-glove/019_0101.jpg


In [23]:
train_df.head(3)

,image_path,p,x,y,w,h
0,/data01/ML/dataset/FACE_CLASSIFIER/256_ObjectC...,0,0,0,0,0
1,/data01/ML/dataset/FACE_CLASSIFIER/256_ObjectC...,0,0,0,0,0
2,/data01/ML/dataset/FACE_CLASSIFIER/256_ObjectC...,0,0,0,0,0


In [24]:
train_datagen = ImageDataGenerator(
    rescale=1./255)
    #preprocessing_function = custom_preprocess_input)

train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=None,
        x_col="image_path",
        y_col=["p","x","y","w","h"],
        class_mode="raw",
        shuffle=True,
        color_mode = 'grayscale',
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE)

valid_generator = train_datagen.flow_from_dataframe(
        dataframe=valid_df,
        directory=None,
        x_col="image_path",
        y_col=["p","x","y","w","h"],
        class_mode="raw",
        color_mode = 'grayscale',
        shuffle=True,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE)

Found 43582 validated image filenames.
Found 7691 validated image filenames.


In [25]:
step_size_train = train_generator.samples/train_generator.batch_size
step_size_valid = valid_generator.samples/valid_generator.batch_size

In [26]:
inp = Input(shape=(IMG_SIZE,IMG_SIZE,1))
darknetv1 = (Conv2D(64,3, strides=(1,1), padding = "same"))(inp)
darknetv1 = (BatchNormalization())(darknetv1)
darknetv1 = (LeakyReLU(alpha=0.1))(darknetv1)
darknetv1 = (MaxPooling2D(pool_size=(2, 2)))(darknetv1)
darknetv1 = (Conv2D(192,3, strides=(1,1), padding = "same"))(darknetv1)
darknetv1 = (BatchNormalization())(darknetv1)
darknetv1 = (LeakyReLU(alpha=0.1))(darknetv1)
darknetv1 = (MaxPooling2D(pool_size=(2, 2)))(darknetv1)

darknetv1 = (Conv2D(128,1, strides=(1,1), padding = "same"))(darknetv1)
darknetv1 = (BatchNormalization())(darknetv1)
darknetv1 = (LeakyReLU(alpha=0.1))(darknetv1)
darknetv1 = (Conv2D(256,3, strides=(1,1), padding = "same"))(darknetv1)
darknetv1 = (BatchNormalization())(darknetv1)
darknetv1 = (LeakyReLU(alpha=0.1))(darknetv1)
darknetv1 = (Conv2D(256,1, strides=(1,1), padding = "same"))(darknetv1)
darknetv1 = (BatchNormalization())(darknetv1)
darknetv1 = (LeakyReLU(alpha=0.1))(darknetv1)
darknetv1 = (Conv2D(512,3, strides=(1,1), padding = "same"))(darknetv1)
darknetv1 = (BatchNormalization())(darknetv1)
darknetv1 = (LeakyReLU(alpha=0.1))(darknetv1)
darknetv1 = (MaxPooling2D(pool_size=(2, 2)))(darknetv1)

darknetv1 = (Conv2D(256,1, strides=(1,1), padding = "same"))(darknetv1)
darknetv1 = (BatchNormalization())(darknetv1)
darknetv1 = (LeakyReLU(alpha=0.1))(darknetv1)
darknetv1 = (Conv2D(512,3, strides=(1,1), padding = "same"))(darknetv1)
darknetv1 = (BatchNormalization())(darknetv1)
darknetv1 = (LeakyReLU(alpha=0.1))(darknetv1)
darknetv1 = (Conv2D(256,1, strides=(1,1), padding = "same"))(darknetv1)
darknetv1 = (BatchNormalization())(darknetv1)
darknetv1 = (LeakyReLU(alpha=0.1))(darknetv1)
darknetv1 = (Conv2D(512,3, strides=(1,1), padding = "same"))(darknetv1)
darknetv1 = (BatchNormalization())(darknetv1)
darknetv1 = (LeakyReLU(alpha=0.1))(darknetv1)
darknetv1 = (Conv2D(256,1, strides=(1,1), padding = "same"))(darknetv1)
darknetv1 = (BatchNormalization())(darknetv1)
darknetv1 = (LeakyReLU(alpha=0.1))(darknetv1)
darknetv1 = (Conv2D(512,3, strides=(1,1), padding = "same"))(darknetv1)
darknetv1 = (BatchNormalization())(darknetv1)
darknetv1 = (BatchNormalization())(darknetv1)
darknetv1 = (LeakyReLU(alpha=0.1))(darknetv1)
darknetv1 = (Conv2D(256,1, strides=(1,1), padding = "same"))(darknetv1)
darknetv1 = (BatchNormalization())(darknetv1)
darknetv1 = (LeakyReLU(alpha=0.1))(darknetv1)
darknetv1 = (Conv2D(512,3, strides=(1,1), padding = "same"))(darknetv1)
darknetv1 = (BatchNormalization())(darknetv1)
darknetv1 = (LeakyReLU(alpha=0.1))(darknetv1)
darknetv1 = (Conv2D(512,1, strides=(1,1), padding = "same"))(darknetv1)
darknetv1 = (BatchNormalization())(darknetv1)
darknetv1 = (LeakyReLU(alpha=0.1))(darknetv1)

darknetv1 = (Conv2D(1024,3, strides=(1,1), padding = "same"))(darknetv1)
darknetv1 = (BatchNormalization())(darknetv1)
darknetv1 = (LeakyReLU(alpha=0.1))(darknetv1)
darknetv1 = (MaxPooling2D(pool_size=(2, 2)))(darknetv1)

darknetv1 = (Conv2D(512,1, strides=(1,1), padding = "same"))(darknetv1)
darknetv1 = (BatchNormalization())(darknetv1)
darknetv1 = (LeakyReLU(alpha=0.1))(darknetv1)
darknetv1 = (Conv2D(1024,3, strides=(1,1), padding = "same"))(darknetv1)
darknetv1 = (BatchNormalization())(darknetv1)
darknetv1 = (LeakyReLU(alpha=0.1))(darknetv1)
darknetv1 = (Conv2D(512,1, strides=(1,1), padding = "same"))(darknetv1)
darknetv1 = (BatchNormalization())(darknetv1)
darknetv1 = (LeakyReLU(alpha=0.1))(darknetv1)
darknetv1 = (Conv2D(1024,3, strides=(1,1), padding = "same"))(darknetv1)
darknetv1 = (BatchNormalization())(darknetv1)
mid_model = (LeakyReLU(alpha=0.1))(darknetv1)

classifier = (MaxPooling2D(pool_size=(2, 2)))(mid_model)
classifier = (Flatten())(classifier)
classifier = (Dense(1024))(classifier)
classifier = (LeakyReLU(alpha=0.1))(classifier)
classifier = (Dropout(0.5))(classifier)
classifier = (Dense(512))(classifier)
classifier = (LeakyReLU(alpha=0.1))(classifier)
classifier  = (Dropout(0.5))(classifier)
classifier = (Dense(1, activation = "sigmoid"))(classifier)

In [27]:
model_temp = Model(inputs=[inp], outputs=classifier)

In [28]:
model_temp.load_weights("face_classifier_BN_GRAYSCALE.h5") 

In [29]:
for layer in model_temp.layers:
    if "dense" in layer.name:
        layer.trainable=True
    else: 
        layer.trainable=False

In [30]:
model_temp.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 1)       0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 224, 224, 64)      640       
_________________________________________________________________
batch_normalization_22 (Batc (None, 224, 224, 64)      256       
_________________________________________________________________
leaky_re_lu_23 (LeakyReLU)   (None, 224, 224, 64)      0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 112, 112, 192)     110784    
_________________________________________________________________
batch_normalization_23 (Batc (None, 112, 112, 192)     768 

# Side tuning time

In [31]:
bounding_boxer = (Conv2D(1024,3, strides=(1,1), padding = "same"))(mid_model)
bounding_boxer = (BatchNormalization())(bounding_boxer)
bounding_boxer = (LeakyReLU(alpha=0.1))(bounding_boxer)
bounding_boxer = (Conv2D(2048,3, strides=(1,1), padding = "same"))(bounding_boxer)
bounding_boxer = (BatchNormalization())(bounding_boxer)
bounding_boxer = (LeakyReLU(alpha=0.1))(bounding_boxer)
bounding_boxer = (Conv2D(1024,3, strides=(1,1), padding = "same"))(bounding_boxer)
bounding_boxer = (BatchNormalization())(bounding_boxer)
bounding_boxer = (LeakyReLU(alpha=0.1))(bounding_boxer)
bounding_boxer = (Conv2D(2048,3, strides=(1,1), padding = "same"))(bounding_boxer)
bounding_boxer = (BatchNormalization())(bounding_boxer)
bounding_boxer = (LeakyReLU(alpha=0.1))(bounding_boxer)
bounding_boxer = (MaxPooling2D(pool_size=(2, 2)))(bounding_boxer)

bounding_boxer = (Flatten())(bounding_boxer)
bounding_boxer = (Dense(1024))(bounding_boxer)
bounding_boxer = (LeakyReLU(alpha=0.1))(bounding_boxer)
bounding_boxer = (Dropout(0.5))(bounding_boxer)

bounding_boxer = (Dense(512))(bounding_boxer)
bounding_boxer = (Dropout(0.5))(bounding_boxer)
bounding_boxer = (LeakyReLU(alpha=0.1))(bounding_boxer)
bounding_boxer = (Dense(4, activation = "relu"))(bounding_boxer)

In [32]:
model = Concatenate()([classifier, bounding_boxer])

In [33]:
model = Model(inputs=[inp], outputs=model)

In [34]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 1)  0                                            
__________________________________________________________________________________________________
conv2d_21 (Conv2D)              (None, 224, 224, 64) 640         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_22 (BatchNo (None, 224, 224, 64) 256         conv2d_21[0][0]                  
__________________________________________________________________________________________________
leaky_re_lu_23 (LeakyReLU)      (None, 224, 224, 64) 0           batch_normalization_22[0][0]     
____________________________________________________________________________________________

In [35]:
model = multi_gpu_model(model,gpus=2)

In [36]:
model.compile(optimizer=Adam(lr = 1e-5), loss=yolf_loss, metrics=[yolf_loss])

In [37]:
#model.evaluate_generator(valid_generator, steps=step_size_valid, verbose = 1)

In [38]:
training_tuples = [(80,1e-4),(40,1e-5),(30,1e-6)]
for tupl in training_tuples:
    print(tupl[0],tupl[1])

80 0.0001
40 1e-05
30 1e-06


In [ ]:
# 80 epoche con 1e-4,40 epoche con 1e-5,30 epoche con 1e-6,
#earlyStopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')
for tupl in training_tuples:
    model.compile(optimizer=Adam(lr = tupl[1]), loss=yolf_loss, metrics=[yolf_loss])
    mcp_save = ModelCheckpoint('steps_tf_training.h5', save_best_only=True, monitor='val_loss', mode='min')
    #reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, mode='min')
    history = model.fit_generator(generator=train_generator, epochs=tupl[0], steps_per_epoch=step_size_train, validation_data=valid_generator, validation_steps=step_size_valid, verbose=1, callbacks=[mcp_save])


Epoch 1/80
341/340 [==============================] - 246s 721ms/step - loss: 8538739.2794 - yolf_loss: 8538716.0000 - val_loss: 3667185.7500 - val_yolf_loss: 7602383.0000
Epoch 2/80
341/340 [==============================] - 242s 709ms/step - loss: 8050808.0502 - yolf_loss: 8050796.0000 - val_loss: 303190.5312 - val_yolf_loss: 7319914.0000
Epoch 3/80
341/340 [==============================] - 233s 684ms/step - loss: 7918866.7106 - yolf_loss: 7918838.0000 - val_loss: 355570.0000 - val_yolf_loss: 7712787.5000
Epoch 4/80
341/340 [==============================] - 226s 662ms/step - loss: 7771925.3231 - yolf_loss: 7771906.5000 - val_loss: 37589.1641 - val_yolf_loss: 7305576.5000
Epoch 5/80
341/340 [==============================] - 230s 676ms/step - loss: 7698793.2493 - yolf_loss: 7698782.0000 - val_loss: 362179.0000 - val_yolf_loss: 7488035.0000
Epoch 6/80
341/340 [==============================] - 234s 686ms/step - loss: 7652664.4612 - yolf_loss: 7652644.0000 - val_loss: 162711.7031 - v

In [ ]:
history.history

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))lu
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training accuracy')
plt.plot(val_acc, label='Validation accuracy')
plt.legend(loc='lower right')
plt.ylabel('accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

# Testing model

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory=None,
        x_col="image_path",
        y_col=["p","x","y","w","h"],
        class_mode="raw",
        shuffle=True,
        color_mode = 'grayscale',
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE)

STEP_SIZE_TEST = test_generator.n / test_generator.batch_size 


#CHANGE PARALLEL MODEL
pred=model.predict_generator(test_generator,  steps=STEP_SIZE_TEST,  verbose=1)

In [ ]:
print(pred)

In [ ]:
count = 0

for el in zip(pred,test_generator.labels,test_generator.filenames):
    count = count +1
    pred_temp = round(el[0][0],2)
    #print(el[0][0])
    if pred_temp >= 0.5:
        print(pred)

        im = np.array(Image.open(el[2]), dtype=np.uint8)

        # Create figure and axes
        fig,ax = plt.subplots(1)

        # Display the image
        ax.imshow(im)

        # Create a Rectangle patch
        rect = patches.Rectangle((50,100),40,30,linewidth=1,edgecolor='r',facecolor='none')
        # Add the patch to the Axes
        ax.add_patch(rect)
        #plt.imshow(img)
        plt.text(0, 0, str(pred_temp), bbox=dict(fill=True, edgecolor='red', linewidth=10))
        plt.savefig("/home/lorenzo.stacchio/ML/Machine_learning_project/dataset/results_total/output"+ str(count) + ".png")
    